In [1]:
from datasets import load_dataset_builder, load_dataset
import json

In [2]:
target_dataset = "ColumbiaNLP/FigLang2022SharedTask"

## Step0: Let's examine the data without committing to download it 

In [3]:
ds_builder = load_dataset_builder(target_dataset)

# Inspect dataset description
print(ds_builder.info.description)

# # Inspect dataset features
print(ds_builder.info.features)

Using custom data configuration ColumbiaNLP--FigLang2022SharedTask-b90ac1c68653c022



{'id': Value(dtype='int64', id=None), 'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None), 'explanation': Value(dtype='string', id=None), 'split': Value(dtype='string', id=None), 'type': Value(dtype='string', id=None), 'idiom': Value(dtype='string', id=None)}


## Step1: Load and preprocess

In [4]:
dataset = load_dataset(target_dataset)

Using custom data configuration ColumbiaNLP--FigLang2022SharedTask-b90ac1c68653c022
Reusing dataset json (/Users/yulinggu/.cache/huggingface/datasets/ColumbiaNLP___json/ColumbiaNLP--FigLang2022SharedTask-b90ac1c68653c022/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'explanation', 'split', 'type', 'idiom'],
        num_rows: 7534
    })
})

In [6]:
set(dataset['train']['label'])

{'Contradiction', 'Entailment'}

In [7]:
def add_combined_cols(entry):
    
    premise = entry["premise"].strip()
    hypothesis = entry["hypothesis"].strip()
    
    if not premise.endswith("."):
        premise += "."
    assert(premise.endswith("."))
    if not hypothesis.endswith("."):
        hypothesis += "."
    assert(hypothesis.endswith("."))
    
    entry["premise_hypothesis"] = 'Premise: ' + premise + ' Hypothesis: ' + hypothesis
    entry["label_explanation"] = 'Label: ' + entry["label"] + '. Explanation: ' + entry["explanation"]
    return entry

In [8]:
# combine columns
combined_cols_dataset = dataset['train'].map(add_combined_cols)

# create train test split because given data has only train data
# splits are shuffled by default
dataset_train_test = combined_cols_dataset.train_test_split(test_size=0.2, seed=42)


Loading cached processed dataset at /Users/yulinggu/.cache/huggingface/datasets/ColumbiaNLP___json/ColumbiaNLP--FigLang2022SharedTask-b90ac1c68653c022/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-3e962b70f0346dce.arrow
Loading cached split indices for dataset at /Users/yulinggu/.cache/huggingface/datasets/ColumbiaNLP___json/ColumbiaNLP--FigLang2022SharedTask-b90ac1c68653c022/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-1b4ed73954418474.arrow and /Users/yulinggu/.cache/huggingface/datasets/ColumbiaNLP___json/ColumbiaNLP--FigLang2022SharedTask-b90ac1c68653c022/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-eba25f89759be04b.arrow


In [9]:
dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'explanation', 'split', 'type', 'idiom', 'premise_hypothesis', 'label_explanation'],
        num_rows: 6027
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'explanation', 'split', 'type', 'idiom', 'premise_hypothesis', 'label_explanation'],
        num_rows: 1507
    })
})

In [10]:
#!mkdir ../figlang_lbv_data

In [11]:
dataset_train_test['train'].to_json("../figlang_lbv_data/0_train_original.json")
dataset_train_test['test'].to_json("../figlang_lbv_data/0_val_original.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1223184